<a href="https://colab.research.google.com/github/Teganmosi/Medical_chatbot/blob/main/medical_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To get started, make sure you remember the following things to ensure the code runs smoothly


*   Ensure you are connected to a GPU before you get started.
*   Ensure you have the [data](https://drive.google.com/drive/folders/15MOFZdq5WWc-VAgo8JPFLOY22YRpMpN7?usp=drive_link) saved in your google drive.

*   Make sure you have your [huggingface access token](https://huggingface.co/settings/tokens) ready to login to huggingface to be able to access the model used for training the chatbot

Once everything is set and ready, Click on the Runtime button and the click on 'Run all' to get started





**Install the Libaries**

In [ ]:
!pip install pypdf
!pip install python-dotenv
!pip install gradio

  Using cached pydantic-2.5.1-py3-none-any.whl (381 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
langchain 0.0.208 requires pydantic<2,>=1, but you have pydantic 2.5.1 which is incompatible.
langchainplus-sdk 0.0.20 requires pydantic<2,>=1, but you have pydantic 2.5.1 which is incompatible.


In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
gradio 4.5.0 requires pydantic>=2.0, but you have pydantic 1.10.13 which is incompatible.


In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install langchain==0.0.208

In [ ]:
!pip install llama-index

**Configure logging and Importing the libaries**

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from langchain.document_loaders import PyPDFLoader


ModuleNotFoundError: ignored

**Mount Google Drive to access data (you may need to authenticate)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Reading the data from the saved path in google drive**

In [ ]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/Data").load_data()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import PyPDFLoader

**Define a system prompt for the Q&A assistant**

In [ ]:

from llama_index.prompts.prompts import SimpleInputPrompt


system_prompt =  "You are a medical chatbot designed to provide general medical information and guidance to patients. You are not a substitute for a licensed healthcare professional. Please always consult with a doctor or other qualified healthcare provider for any medical concerns. Here are some specific examples of how you can be helpful: Answer general medical questions about symptoms, conditions, and treatments, Provide information on healthy lifestyle habits and preventive care, Help patients find reliable medical resources and support groups, Offer emotional support and reassurance to patients facing medical challenges. Please keep in mind the following: Always direct patients to seek professional medical attention, Use clear and concise language that is easy for patients to understand. Be respectful and sensitive to patients individual needs and concerns.I am confident that you can be a valuable tool for patients seeking medical information and guidance. Thank you for your dedication to helping others."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

**Log in to Hugging Face**

In [ ]:
#!huggingface-cli login

**Configure the HuggingFaceLLM (Language Model)**

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.5, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="NousResearch/Llama-2-7b-chat-hf",
    model_name="NousResearch/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True})

**Configure embeddings using Hugging Face model**

In [ ]:
pip install llama_index

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index import ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


**Configure the service context**

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

**Create a vector store index from the loaded documents**

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

**Create a query engine for the index**

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What  are several common symptoms of cataracts?")

In [ ]:
print(response)

**To interact with the chatbot**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install streamlit


In [ ]:
import streamlit as st

In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

In [ ]:
import gradio as gr

ModuleNotFoundError: ignored